In [55]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression, chi2, mutual_info_classif
import matplotlib.pyplot as plt

## Dados escolares

In [56]:
#abrindo os dados
dados_escolares = pd.read_csv('DadosEscolares1996-2015.csv', encoding = 'latin1')
nota = pd.read_csv('ENEM2015.csv')
#agrupando por escolas
dados_escolares = dados_escolares.groupby('CODMEC').mean()
#Apagando a classe desbalanceada
nota = nota[nota.Classificacao != 0]
#Em todos Dataframes temos mais escolas do que o nosso target
dados_escolares = dados_escolares.reset_index()[dados_escolares.reset_index().CODMEC.isin(nota['CODIGO DA ENTIDADE'])]
dados_escolares.drop(dados_escolares.columns[dados_escolares.isnull().mean() != 0], axis=1, inplace = True)
#Alterando o nome da coluna para o merge
nota.rename(columns = {'CODIGO DA ENTIDADE':'CODMEC'}, inplace = True)
#criando o merge com base na coluna CODMEC usando o dataframe target e os dados escolares
new_df = pd.merge(dados_escolares, nota, on = 'CODMEC')
new_df.drop(['CODMEC', 'ANO','NUMERO DE PARTICIPANTES'], axis = 1, inplace = True)
#criando o data e o target
data = new_df.drop('Classificacao', axis = 1)
target = new_df.Classificacao
print(data.shape)

C:\Users\giova\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6,8,11,17,18,19,20,111,240) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(692, 210)


### f regression

In [57]:
#Rodando o selectKBest para 12 colunas
melhor_score = 0
melhores_colunas = []
for var in list(range(1,45)):
    new_data = SelectKBest(f_regression, k=var).fit_transform(data, target)
    print(new_data.shape)
    lm = LinearRegression()
    scaler = StandardScaler()
    data1 = scaler.fit_transform(new_data)
    kf = KFold(5, shuffle = True)
    r = []
    for train_i, test_i in kf.split(data1):
        x_train, x_test = data1[train_i], data1[test_i]
        y_train, y_test = target[train_i], target[test_i]
        lm.fit(x_train, y_train)
        r.append(lm.score(x_test, y_test))
    print('Score:',sum(r)/5)
    if melhor_score < sum(r)/5:
        melhor_score = sum(r)/5
        melhores_colunas = data.columns[kbest.get_support()]

(692, 1)
Score: 0.19984315412112058


C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
C:\Users\giova\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\giova\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWar

NotFittedError: This SelectKBest instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [53]:
print('O melhor score foi: ', melhor_score)
print('Com as seguintes colunas: \n', melhores_colunas)
len(melhores_colunas)

O melhor score foi:  0
Com as seguintes colunas: 
 []


0

### chi 2

In [58]:
#Rodando o selectKBest para 12 colunas
melhor_score = 0
melhores_colunas = []
for var in list(range(1,45)):
    kbest = SelectKBest(chi2, k = var)
    new_data = kbest.fit_transform(data, target)
    print(new_data.shape)
    lm = LinearRegression()
    scaler = StandardScaler()
    data1 = scaler.fit_transform(new_data)
    kf = KFold(5, shuffle = True)
    r = []
    for train_i, test_i in kf.split(data1):
        x_train, x_test = data1[train_i], data1[test_i]
        y_train, y_test = target[train_i], target[test_i]
        lm.fit(x_train, y_train)
        r.append(lm.score(x_test, y_test))
    print('Score:',sum(r)/5)
    if melhor_score < sum(r)/5:
        melhor_score = sum(r)/5
        melhores_colunas = data.columns[kbest.get_support()]

(692, 1)
Score: -0.020735552402415935
(692, 2)
Score: 0.09254023541263408
(692, 3)
Score: 0.1026193881515904
(692, 4)
Score: 0.16287188735837907
(692, 5)
Score: 0.18902009555531804
(692, 6)
Score: 0.1870235406078205
(692, 7)
Score: 0.19416603010637357
(692, 8)
Score: 0.19122305973507286
(692, 9)
Score: 0.22864979032541816
(692, 10)
Score: 0.25606901804429005
(692, 11)
Score: 0.2463366279572828
(692, 12)
Score: 0.2593170371026665
(692, 13)
Score: 0.3613191462251836
(692, 14)
Score: 0.3769902298212702
(692, 15)
Score: 0.37843622581775277
(692, 16)
Score: 0.3495143566311431
(692, 17)
Score: 0.36167738842125996
(692, 18)
Score: 0.355156634626521
(692, 19)
Score: 0.3821195957956733
(692, 20)
Score: 0.3643143049327343
(692, 21)
Score: 0.34748959765645693
(692, 22)
Score: 0.3434253449194958
(692, 23)
Score: 0.37221647606718083
(692, 24)
Score: 0.36006140945441995
(692, 25)
Score: 0.3115527004385297
(692, 26)
Score: 0.3733318973908431
(692, 27)
Score: 0.35075678336864724
(692, 28)
Score: -0.14

In [59]:
print('O melhor score foi: ', melhor_score)
print('Com as seguintes colunas: \n', melhores_colunas)

O melhor score foi:  0.4291661705108183
Com as seguintes colunas: 
 Index(['Qtde_Docentes', 'NUM_FUNC', 'SETEDU', 'AL_PRE', '5', '6', '6fund',
       '0A4', '4A6', '5A6', 'TotalEdInf', 'ALEM1S', 'ALEM2S', 'ALEM3S', 'ALEM',
       'ALEF4S', 'ALEF5S', 'ALEF6S', 'ALEF7S', 'ALEF8S', 'ALEF1A4', 'ALEF5A8',
       'ALEF1A8', 'ALE9F8S', 'ALE9F9S', 'ALE9F1A5', 'ALE9F6A9', 'ALE9F1A9',
       'ALFUNDI', 'ALFUNDII', 'ALFUND', 'ALEJAMED', 'TOTALEJA', 'ALEDPROF',
       'TOTAL_ALUNOS'],
      dtype='object')


In [60]:
len(melhores_colunas)

35